In [ ]:
from tabulate import tabulate
from string import ascii_uppercase 
from random import choice, shuffle  
from functools import reduce
from itertools import product
import pandas as pd

#Cosas a mejorar
"""
1- IA de los disparos
2- Adecentar codigo del run. Los prints lo hacen dificil de leer.
3- Crear Registro de los barcos con posiciones a la hora de Posicionarlos.
4- Unificar PosicionarCrucerosIA con PosicionarCrucerosPlayer
5- Crear metodo para resetear a 0 DataFrames
6- Enseñar por pantalla el estado de la flota del jugador.
7- Mover al init ALastShot, PlayerLastShot y DisparosPendientes
8- Correciones al introducir mal los inputs.
"""

class Battleship():
    def __init__(self, matrix_size=5 ,boat_size=3,fleet_size=3):
        self.matrix_size = matrix_size
        self.boat_size = boat_size        #Siempre ha de ser impar de la contrario la logica de PosicionarEnTablero no es correcta
        self.fleet_size = fleet_size
        
        self.Player_win_condition = 0
        self.IA_win_condition = 0
        
    def run(self):
        #Inicializamos los DataFrame de las Flotas con una matriz de 0s.
        FlotaIA = self.CreateDataFrame([0])   
        FlotaPlayer = self.CreateDataFrame([0])
        
        #Le doy un DataFrame vacio y me retorna el DataFrame con una Flota posicionada al azar. 
        FlotaIA = self.PosicionarCrucerosIA(FlotaIA) 
        #Pregunta al usuario donde quiere sus cruceros y lo retorna.Si los coloca mal te hace colocarlos otra vez.
        FlotaPlayer = self.PosicionarCrucerosPlayer(FlotaPlayer)   
        
        #Mapa con la informacion que tiene el jugador sobre la flota de la IA.
        RegistroPlayer = self.CreateDataFrame(['']) 
        #Me guardo una copia de mi flota para crear un mapa de los disparos de la IA al Jugador.
        CopiaFlotaPlayer = FlotaPlayer
        
        #Lista con todos los disparos que la IA aun no ha hecho.
        BolsaDisparosRandomIA = self.AllPositions() 
        shuffle(BolsaDisparosRandomIA) #Como de aqui saldran las coordenadas para disparar lo mezclamos para aleatorizarlo.       
        BolsaDisparosPendientes = [] #Bolsa donde se guardan las posiciones que rodeen los aciertos de la IA.
        
        #Inicializamos las variables que almacenan las cordenadas de los últimos disparos.
        IALastShot = (0,0,None) 
        PlayerLastShot = (0,0,None)
        
        #Empieza la partida.
        while self.WinCondition(): #Chequeamos si ha ganado el Jugador.

            #Turno IA
            print('Empieza el turno de la IA.')
            
            #La IA dependiendo del resultado de su ultimo disparo decide donde disparar.
            CoordenadasDisparoIA, BolsaDisparosPendientes = self.IAShooting(BolsaDisparosRandomIA, IALastShot, BolsaDisparosPendientes) 
            print(f'La IA ha decidido disparar a {CoordenadasDisparoIA}')
            
            #Se mira si el disparo de la IA acierta a la Flota del jugador.
            IALastShot = self.ResultadoDisparo(FlotaPlayer,'La IA', CoordenadasDisparoIA) 
            #Actualizamos la condicion de victoria si ha habido un acierto de la IA.
            self.UpdateWinCondition(FlotaPlayer,'La IA', CoordenadasDisparoIA) 
            
            if not self.WinCondition(): #Chequeamos si ha ganado la IA.
                break

            #Turno Jugador
            print('Empieza el turno del Jugador.')
            
            #Imprimimos el Estado de la Flota del jugador con los aciertos y fallos de la IA.
            self.EstadoFlotaPlayer(CopiaFlotaPlayer, IALastShot)
            #Se Actualiza el Mapa con la información de la flota enemiga y se imprime por pantalla.
            self.EstadoRegistroPlayer(RegistroPlayer, PlayerLastShot)
            
            #El Jugador mediante inputs introduce donde quiere disparar.
            CoordenadasDisparoPlayer = self.PlayerShooting()
            print(f'El Jugador ha decidido disparar a {CoordenadasDisparoPlayer}')
            
            #Se mira si el disparo del Jugador ha acertado a la Flota de la IA.
            PlayerLastShot = self.ResultadoDisparo(FlotaIA,'El Jugador', CoordenadasDisparoPlayer)
            #Actualizamos la condicion de victoria si ha habido un acierto del Jugador.
            self.UpdateWinCondition(FlotaIA,'El Jugador', CoordenadasDisparoPlayer) 
        
    def CreateDataFrame(self, lista): #Se crea un DataFrame vacio en el que las columnas estan nombradas en base a letras.
        df = pd.DataFrame([lista*self.matrix_size]*self.matrix_size, columns=list(ascii_uppercase[:self.matrix_size])) 
        df.index += 1  #Forzamos que los indices empiezcen en 1.
        return df 
    
    def PrintDataFrame(self, df):
        print(tabulate(df, headers=df.columns, showindex="always",tablefmt="grid",numalign="center",stralign="center"))
    
    def ComparingFleetSize(self,df):     
        sum=0
        for col in df.values.tolist():
            sum += col.count('X')
        return sum != self.fleet_size*self.boat_size
    
    def PosicionarEnTablero(self, df, Orientation, x, y):
        if Orientation == 'v':
            df.loc[x-self.boat_size//2:x + self.boat_size//2, y] = 'X'
        elif Orientation == 'h':
            df.loc[x, ascii_uppercase[ascii_uppercase.find(y)-self.boat_size//2] : ascii_uppercase[ascii_uppercase.find(y)+self.boat_size//2] ] = 'X'        

    def PosicionarCrucerosIA(self, df ): #Coloca los cruceros al azar en la matriz
        while self.ComparingFleetSize(df): #Hasta que el numero de cruceros no sea igual al de Xs no termina este bucle.
            df = pd.DataFrame(0,columns=df.columns,index=df.index) #Si se han introducido mal y el bucle se repite, resetea df a 0.
            counter = self.fleet_size #Cantidad de cruceros a posicionar. Hasta que no se hayan puesto todos no acaba el while.
            while counter > 0:
                Orientation = choice(['h','v']) #Elige una Orientation random  
                x,y = choice(self.GeneratePosicitions(Orientation)) #Genera una posición random para colocar el crucero.
                self.PosicionarEnTablero(df, Orientation,x,y)#Coloca el crucero en las coordenadas x,y con la orientacion dada.
                counter -= 1 #Resta un crucero del contador
        return df
    
    def PosicionarCrucerosPlayer(self, df, first = True): #Esta función pregunta al usuario donde quiere sus cruceros.
        while self.ComparingFleetSize(df): #Hasta que el numero de cruceros no sea igual al de Xs en df te obliga a colocar cruceros.
            if first: #Escribe este mensaje solo en la primer iteracion del bucle.
                print('Coloca tu flota!')
                first = False 
            elif not first: #En el resto de iteraciones escribiraeste mensaje.
                print('Vaya parece que no la has colocado del todo bien. Intentalo otra vez!')
                
            df = pd.DataFrame(0,columns=df.columns,index=df.index) #Si se han introducido mal y el bucle se repite, resetea df a 0.
            self.PrintDataFrame(df) #Imprimimos la matriz vacia

            counter = self.fleet_size #Cantidad de cruceros a posicionar. Hasta que no se hayan puesto todos no acaba el while.
            while counter > 0:
                Orientation = input("Introduce la orientación de tu crucero. Vertical 'v' o Horizontal 'h'")
                x, y = input(f"Introduce las coordenadas de tu crucero, desde su punto medio. Ten en cuenta que el tamaño de los cruceros es de {self.boat_size} y el formato de coordenadas es de: 2 D").split() 
                self.PosicionarEnTablero(df,Orientation,int(x),y) #Coloca un crucero en la posicion x,y con la Orientacion indicada.
                self.PrintDataFrame(df) #Despues de colocar un crucero imprimimos a ver como queda.
                counter -= 1  #Resta un crucero del contador
        return df
            
    def AllPositions(self): #Retorna una lista de tuples con todas las posiciones de la matriz
        return list(product(range(1,self.matrix_size+1),list(ascii_uppercase[:self.matrix_size])))   
    
    def QuitarPosiciones(self, ColumnasoFilas):
        return list(filter(lambda y: reduce(lambda a,b: a and b, map(lambda x:x not in ColumnasoFilas, y)), self.AllPositions()))

    def GeneratePosicitions(self, Orientation): 
        if Orientation == 'v': 
            RowsToRemove = list(range(1,self.matrix_size+1)[:(self.boat_size//2)]) + list(range(1,self.matrix_size+1)[self.matrix_size-(self.boat_size//2):])
            return self.QuitarPosiciones(RowsToRemove) #Quitamos filas arriba y abajo tal que tamaño_crucero//2

        elif Orientation == 'h':
            ColumnsToRemove = list(ascii_uppercase[:self.matrix_size][:(self.boat_size//2)] + ascii_uppercase[:self.matrix_size][self.matrix_size-(self.boat_size//2):])
            return self.QuitarPosiciones(ColumnsToRemove) #Quitamos columnas del principio y del final tal que tamaño_crucero//2 

    def ResultadoDisparo(self, DataFrame,string,CoordenadasDisparo):
        if DataFrame.loc[CoordenadasDisparo[0],CoordenadasDisparo[1]] == 'X':
            print(f'{string} ha acertado!')
            return CoordenadasDisparo[0], CoordenadasDisparo[1], True
        else:    
            print(f'{string} ha fallado!')
            return CoordenadasDisparo[0], CoordenadasDisparo[1], False

    def IAShooting(self, BolsaDisparosRandomIA, IALastShot, BolsaDisparosPendientes):

        if IALastShot[2] == True: #Si en el ultimo disparo se acerto almacena las 4 posiciones de alrededor en el Registro de Disparos Pendientes
            PosicionesPotenciales = [(IALastShot[0]+1 ,IALastShot[1]),(IALastShot[0]-1 ,IALastShot[1]),
                                    (IALastShot[0],ascii_uppercase[ascii_uppercase.find(IALastShot[1])+1]),
                                    (IALastShot[0],ascii_uppercase[ascii_uppercase.find(IALastShot[1])-1])]
            PosicionesFueraDeRango = [0,self.matrix_size+1,'Z',ascii_uppercase[self.matrix_size+1]]

            BolsaDisparosPendientes += PosicionesPotenciales

            aux = list(set(BolsaDisparosPendientes)).copy() #Copia no ligada a DisparosPendientes, de lo contrario modificar aux modificaria DisparosPendientes
            for cords in BolsaDisparosPendientes: #Filtramos las posiciones que ya se han disparado y los que estan fuera de rango 
                if cords not in BolsaDisparosRandomIA and cords in aux: #Quitamos los disparos que ya se hayan hecho
                    aux.remove(cords)
                for rowcol in PosicionesFueraDeRango:        #Quitamos lo que este fuera de rango de nuestra matriz
                    if rowcol in cords and cords in aux: 
                        aux.remove(cords)
            BolsaDisparosPendientes = aux

        if BolsaDisparosPendientes != []: #Si hay algun diparo pendiente, dispara a una de estas posiciones
            Shot = choice([BolsaDisparosPendientes]).pop() #Hacemos un disparo random de DisparamosPendientes y lo borramos de DisparamosPendientes
            BolsaDisparosRandomIA.remove(Shot) #Lo borramos tambien de la bolsa de disparos de la IA
            return Shot, BolsaDisparosPendientes
        else:                        #Si no hay disparos pendientes dispara al azar  
            return BolsaDisparosRandomIA.pop(), BolsaDisparosPendientes  
        
    def PlayerShooting(self):
        while True:
            try:
                x,y = input(f"Introduce las coordenadas a las que deseas disparar. Ten en cuenta que el tamaño de los cruceros es de {self.boat_size} y el formato de coordenadas es de: 2 D").split()

                if len(x)!=1 or not(isinstance(y, str) and len(y)==1 and y in ascii_uppercase):
                    raise ValueError
                else:
                    return int(x),y
            except:
                print('Esas coordenadas no son correctas. Introducelas otra vez.')
    
    def EstadoFlotaPlayer(self,DataFrame, IALastShot):
        if IALastShot[2]  == True: #Si en el ultimo disparo le hemos dado a un crucero apuntar la posicion con X.
            DataFrame.loc[IALastShot[0],IALastShot[1]] = '\033[1m'+'O'+ "\033[0m"
        elif IALastShot[2]  == False:#Si en el ultimo disparo le hemos dado al agua apuntar lo con un 0.     
            DataFrame.loc[IALastShot[0],IALastShot[1]] = '\033[1m'+'0'+ "\033[0m"
        print('Este es el estado de tu flota.')
        self.PrintDataFrame(DataFrame) 
    
    def EstadoRegistroPlayer(self,DataFrame, PlayerLastShot):
        if PlayerLastShot[2]  == True: #Si en el ultimo disparo le hemos dado a un crucero apuntar la posicion con X.
            DataFrame.loc[PlayerLastShot[0],PlayerLastShot[1]] = 'X'
        elif PlayerLastShot[2]  == False:#Si en el ultimo disparo le hemos dado al agua apuntar lo con un 0.     
            DataFrame.loc[PlayerLastShot[0],PlayerLastShot[1]] = 0
        print('Este es tu registro de la flota enemiga.')
        self.PrintDataFrame(DataFrame)    

    def UpdateWinCondition(self, df, string, CoordenadasDisparo):
        if df.loc[CoordenadasDisparo[0],CoordenadasDisparo[1]] == 'X':
            if string == 'La IA':
                self.IA_win_condition += 1
            elif string == 'El Jugador':
                self.Player_win_condition += 1   
            
    def WinCondition(self): #El primero en acertar boat_size*fleet_Size veces gana
        if self.Player_win_condition == self.boat_size*self.fleet_size:
            print('El Jugador ha ganado!')
            return False
        elif self.IA_win_condition == self.boat_size*self.fleet_size:
            print('La IA ha ganado!')
            return False 
        else:
            return True                  
            
"""def main():
    Battleship().run()
if __name__ == "__main__":
    main()"""

In [ ]:
Partida = Battleship()

Partida.run()